In [1]:
# SampleSAT implementation + usage demo

import random
from dataclasses import dataclass
from typing import Dict, Tuple, Set, List, Optional

from sympy import symbols, And, Or, Not
from sympy.logic.boolalg import to_cnf


# --------- utilities on SymPy formulas ----------

def _vars_set(F) -> Set:
    return set(F.free_symbols)

def _eval_under(F, assign: Dict) -> bool:
    return bool(F.subs(assign))

def _random_assignment(V: Set, rng: Optional[random.Random] = None) -> Dict:
    rng = rng or random
    return {v: bool(rng.getrandbits(1)) for v in V}

def _cnf_clauses(F) -> List[List]:
    """Return CNF as list of clauses (each clause is a list of literals)."""
    cnf = to_cnf(F, simplify=True)
    if cnf == True:
        return []          # no clauses => tautology
    if cnf == False:
        return [[]]        # empty clause => contradiction
    if isinstance(cnf, Or):
        return [[arg for arg in cnf.args]]
    if isinstance(cnf, And):
        clauses = []
        for c in cnf.args:
            if isinstance(c, Or):
                clauses.append([arg for arg in c.args])
            else:
                clauses.append([c])
        return clauses
    # single literal / symbol
    return [[cnf]]

def _is_lit_true(lit, A: Dict) -> bool:
    if lit.func == Not:
        return not bool(A[lit.args[0]])
    return bool(A[lit])

def _unsat_clauses(clauses, A: Dict) -> List[int]:
    """Return indices of currently unsatisfied clauses."""
    bad = []
    for i, clause in enumerate(clauses):
        if not any(_is_lit_true(lit, A) for lit in clause):
            bad.append(i)
    return bad

def _flip_var(A: Dict, v) -> None:
    A[v] = not A[v]


# --------- SampleSAT (Selman, Kautz, Cohen '94; Kautz & Selman '96) ----------

@dataclass
class SampleSATResult:
    sat: bool
    assignment: Dict
    tries: int
    flips: int

def sample_sat(F, max_tries: int = 50, max_flips: int = 1000, noise: float = 0.5, seed: Optional[int] = None) -> SampleSATResult:
    """
    Stochastic local search con random-walk (WalkSAT) + mossa greedy (GSAT-style).
    - noise = probabilità di scegliere una mossa casuale su una clausola insoddisfatta.
    Ritorna: (sat, assegnamento, tries usati, flips usati)
    """
    rng = random.Random(seed)
    V = _vars_set(F)
    clauses = _cnf_clauses(F)

    # Casi banali
    if clauses == []:   # tautologia
        A = _random_assignment(V, rng)
        return SampleSATResult(True, A, 0, 0)
    if clauses == [[]]: # contraddizione
        return SampleSATResult(False, {}, 0, 0)

    total_flips = 0
    for t in range(1, max_tries + 1):
        A = _random_assignment(V, rng)
        for f in range(1, max_flips + 1):
            if _eval_under(F, A):
                return SampleSATResult(True, dict(A), t, total_flips + f - 1)

            bad_ids = _unsat_clauses(clauses, A)

            if rng.random() < noise:
                # RANDOM WALK: prendo a caso una clausola insoddisfatta e flippo una sua variabile a caso
                clause = clauses[rng.choice(bad_ids)]
                # estraggo la variabile "grezza" dal letterale (X oppure Not(X))
                lits_vars = [lit.args[0] if lit.func == Not else lit for lit in clause]
                v = rng.choice(lits_vars)
                _flip_var(A, v)
            else:
                # MOSSA GREEDY: scegli la variabile che minimizza il numero di clausole insoddisfatte dopo il flip
                best_v = None
                best_score = None
                for v in V:
                    _flip_var(A, v)
                    score = len(_unsat_clauses(clauses, A))
                    _flip_var(A, v)  # ripristina
                    if best_score is None or score < best_score:
                        best_score = score
                        best_v = v
                _flip_var(A, best_v)

        total_flips += max_flips

    return SampleSATResult(False, {}, max_tries, total_flips)


# --------- Stima #SAT facoltativa (molto grezza, solo dimostrativa) ----------
def estimate_count_samplesat(F, runs: int = 200, **kwargs) -> Tuple[int, float]:
    """
    Esegue più run di SampleSAT e stima #SAT ~= p_hat * 2^n,
    dove p_hat = frazione di run che trovano un modello entro il budget.
    NOTA: è una stima *molto* rozza e dipende pesantemente dal budget di ricerca.
    """
    V = _vars_set(F)
    n = len(V)
    successes = 0
    for _ in range(runs):
        res = sample_sat(F, **kwargs)
        successes += int(res.sat)
    p_hat = successes / runs if runs else 0.0
    est = int(round(p_hat * (2 ** n)))
    return est, p_hat


# -------------------- DEMO DI UTILIZZO --------------------

if __name__ == "__main__":
    # Variabili booleane
    A, B, C, D = symbols('A B C D')

    # Una formula non banale (implicazioni e vincoli incrociati)
    F = And(
        Or(A, B),               # A ∨ B
        Or(B, C),               # B ∨ C
        Or(Not(A), C),          # ¬A ∨ C
        Or(Not(B), D),          # ¬B ∨ D
        Or(Not(C), D),          # ¬C ∨ D
    )

    print("Formula F:", F)

    # 1) Ricerca di un'assegnazione soddisfacente
    res = sample_sat(F, max_tries=60, max_flips=1000, noise=0.45, seed=42)
    print("\n--- Risultato SampleSAT ---")
    print("Soddisfacibile?:", res.sat)
    print("Assegnamento trovato:", res.assignment)
    print("Tries usati:", res.tries, " | Flips totali:", res.flips)
    print("Verifica (F sotto assegnamento):", bool(F.subs(res.assignment)) if res.sat else None)

    # 2) (Facoltativo) Stima grezza del numero di modelli
    est, p_hat = estimate_count_samplesat(F, runs=200, max_tries=40, max_flips=400, noise=0.5)
    print("\n--- Stima #SAT (dimostrativa) ---")
    print("#vars:", len(_vars_set(F)), "| p_hat:", f"{p_hat:.3f}", "| stima ≈", est)

Formula F: (A | B) & (B | C) & (C | ~A) & (D | ~B) & (D | ~C)

--- Risultato SampleSAT ---
Soddisfacibile?: True
Assegnamento trovato: {C: True, A: True, B: False, D: True}
Tries usati: 1  | Flips totali: 1
Verifica (F sotto assegnamento): True

--- Stima #SAT (dimostrativa) ---
#vars: 4 | p_hat: 1.000 | stima ≈ 16
